## NBA data web crawling

In [55]:
#載入需要套件
import requests
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import lxml

from collections import OrderedDict

##### 寫一個函式將 basketball-reference.com 中球員的表現數據 爬蟲下來 存入回傳dataframe

In [56]:
# function converts string to float
def convert_numeric(df, str_cols):
    for i in str_cols:
        df[i] = df[i].astype(float)
        
def scrape_per_min_stats(year):
    # main url - change year parameter to get different years
    url = 'https://www.basketball-reference.com/leagues/NBA_'+ str(year) + '_per_minute.html'
    response = requests.get(url)
    page = response.text
    
    soup = BeautifulSoup(page, "html.parser") 
    
    # grab correct col names
    cols =  soup.find('thead').text.split('\n')
    cols_clean = cols[3:-2]
    
    # start to collect data from table to list of lists
    table = soup.find('tbody')
    rows = table.find_all('tr')

    data1 = []
    for tr in rows:
        cols = tr.find_all('td')
        x = [td.text if td.getText() != '' else '0' for td in cols]
        data1.append(x)
        
    # convert to pandas DF
    Table = pd.DataFrame(data=data1, columns=cols_clean)
    
    # string cols to convert
    str_cols = [Table.columns[2]] + Table.columns[4:].tolist()

    # drop NAs, reset index
    Table.dropna(how='all', inplace=True)
    Table.reset_index(drop=True, inplace=True)
    
    # convert cols to numeric
    convert_numeric(Table, str_cols)
    
    # drop duplicates
    Table1 = Table.sort_values('G', ascending=False).drop_duplicates('Player').sort_index()
    Table1['Year'] = year

    return Table1


def scrape_advanced_stats(year):
    url = 'https://www.basketball-reference.com/leagues/NBA_' + str(year) + '_advanced.html'
    
    soup = BeautifulSoup(requests.get(url).text, "html.parser")
    table = soup.find('tbody')
    
    col_names = ['Player', 'Pos', 'Age', 'Tm', 'G', 'MP', 'PER', 'TS%', '3PAr', 'FTr',\
          'ORB%', 'DRB%', 'TRB%', 'AST%', 'STL%', 'BLK%', 'TOV%', 'USG%', 'DONOTUSE1',\
          'OWS', 'DWS', 'WS', 'WS/48', 'DONOTUSE2','OBPM', 'DBPM', 'BPM', 'VORP']
    
    # convert to list of lists
    rows = table.find_all('tr')

    data = []
    for tr in rows:
        cols = tr.find_all('td')
        x = [td.text if td.getText() != '' else '0' for td in cols]
        data.append(x)
       
    # convert to DF and add col names
    data_adv = pd.DataFrame(data=data, columns=col_names)
    
    # drop NAs, reset index
    data_adv.dropna(how='all', inplace=True)
    data_adv.reset_index(drop=True, inplace=True)
    
    # convert string cols to numeric
    str_cols = [data_adv.columns[2]] + data_adv.columns[4:].tolist()
    convert_numeric(data_adv, str_cols)

    # dedup - This happens if a player was on multiple teams in the same season
    data_adv = data_adv.sort_values('G', ascending=False).drop_duplicates('Player').sort_index()
    
    # drop unneeded cols
    data_adv = data_adv.drop(['Pos','Age', 'Tm', 'G', 'MP', 'DONOTUSE1', 'DONOTUSE2'], 1)

    return data_adv

##### 用上面函式 將2013~2019年球員的表現數據 爬蟲下來 存入回傳並合併dataframe

In [57]:
# Create a dictionary of data for 2013-2019 seasons
# Merge per-min stats and advanced stats
year_start = 2013
year_end = 2019

print('Scraping Per Minute Stats and Advanced Stats')
print('Seasons', year_start, 'through', year_end, '\n')

years = range(year_start, year_end+1, 1)
data_stats = OrderedDict()

for year in years:
    print('Scraping', year)
    _df_per_min = scrape_per_min_stats(year)
    _df_advanced = scrape_advanced_stats(year)
    _df_merged = pd.merge(_df_per_min, _df_advanced, how='left', on=['Player'])
    _df_merged['Year'] = year
    print('Number of unique players', _df_merged.Player.nunique(), '\n')
    data_stats[year] = _df_merged

# Concat All DFs
df_all = pd.concat([df for year, df in data_stats.items()], ignore_index=True)
print('Done!')

Scraping Per Minute Stats and Advanced Stats
Seasons 2013 through 2019 

Scraping 2013
Number of unique players 468 

Scraping 2014
Number of unique players 481 

Scraping 2015
Number of unique players 492 

Scraping 2016
Number of unique players 476 

Scraping 2017
Number of unique players 486 

Scraping 2018
Number of unique players 540 

Scraping 2019
Number of unique players 530 

Done!


In [58]:
df_all.head()

,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,...,TOV%,USG%,OWS,DWS,WS,WS/48,OBPM,DBPM,BPM,VORP
0,Quincy Acy,PF,22.0,TOR,29.0,0.0,342.0,4.4,7.9,0.560,...,15.6,14.7,0.7,0.4,1.1,0.157,-0.6,1.3,0.7,0.2
1,Jeff Adrien,PF,26.0,CHA,52.0,5.0,713.0,3.6,8.5,0.429,...,13.1,15.6,0.5,0.4,1.0,0.064,-3.0,-0.2,-3.3,-0.2
2,Arron Afflalo,SF,27.0,ORL,64.0,64.0,2307.0,6.2,14.1,0.439,...,12.1,22.5,1.5,0.5,2.0,0.042,-0.3,-2.0,-2.3,-0.2
3,Josh Akognon,PG,26.0,DAL,3.0,0.0,9.0,8.0,16.0,0.500,...,0.0,20.3,0.0,0.0,0.0,0.196,4.7,-4.9,-0.2,0.0
4,Cole Aldrich,C,24.0,TOT,45.0,0.0,388.0,4.1,7.4,0.550,...,20.6,12.7,0.1,0.4,0.6,0.070,-4.8,0.7,-4.0,-0.2


In [59]:
df_all.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3473 entries, 0 to 3472
Data columns (total 49 columns):
Player    3473 non-null object
Pos       3473 non-null object
Age       3473 non-null float64
Tm        3473 non-null object
G         3473 non-null float64
GS        3473 non-null float64
MP        3473 non-null float64
FG        3473 non-null float64
FGA       3473 non-null float64
FG%       3473 non-null float64
3P        3473 non-null float64
3PA       3473 non-null float64
3P%       3473 non-null float64
2P        3473 non-null float64
2PA       3473 non-null float64
2P%       3473 non-null float64
FT        3473 non-null float64
FTA       3473 non-null float64
FT%       3473 non-null float64
ORB       3473 non-null float64
DRB       3473 non-null float64
TRB       3473 non-null float64
AST       3473 non-null float64
STL       3473 non-null float64
BLK       3473 non-null float64
TOV       3473 non-null float64
PF        3473 non-null float64
PTS       3473 non-null float64


In [60]:
df_all.columns

Index(['Player', 'Pos', 'Age', 'Tm', 'G', 'GS', 'MP', 'FG', 'FGA', 'FG%', '3P',
       '3PA', '3P%', '2P', '2PA', '2P%', 'FT', 'FTA', 'FT%', 'ORB', 'DRB',
       'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', 'Year', 'PER', 'TS%',
       '3PAr', 'FTr', 'ORB%', 'DRB%', 'TRB%', 'AST%', 'STL%', 'BLK%', 'TOV%',
       'USG%', 'OWS', 'DWS', 'WS', 'WS/48', 'OBPM', 'DBPM', 'BPM', 'VORP'],
      dtype='object')

In [61]:
df_all.shape

(3473, 49)

In [62]:
df_all.Year.unique()

array([2013, 2014, 2015, 2016, 2017, 2018, 2019])

In [63]:
df_all.Player.nunique()

1041

In [64]:
len(df_all.Player.unique())

1041

##### 寫一個函式將 espn.com 中球員的薪水 爬蟲下來 存入回傳dataframe

In [65]:
def scrape_espn_contract(season, pages):
    ### extract name
    def get_name(col):
        return(col.split(",",1)[0])

    # base url
    url_espn ='http://www.espn.com/nba/salaries/_/year/'
    # Example: http://www.espn.com/nba/salaries/_/year/2017/page/2/seasontype/1
    # create list of urls, since they span multiple pages
    # this list comprehension takes 'SEASON' and 'PAGES' parameters
    url_list = [url_espn + str(season) + '/page/' + str(i) + '/seasontype/1' for i in range(1, pages + 1)]
    
    appended_data = []
    
    # cycle through pages
    for url in url_list:       
        soup = BeautifulSoup(requests.get(url).text, "html.parser")
        rows = soup.find_all('tr')
        col_names = ['NAME', 'TEAM','SALARY']

        data = []
        for tr in rows[1:]:
            cols = tr.find_all('td')[1:]
            x = [td.text for td in cols if td.text not in col_names]
            data.append(x)

        Table = pd.DataFrame(data)
        # drop NAs
        Table.dropna(how='all', inplace=True)
        Table.reset_index(drop=True, inplace=True)
        # add col names
        Table.columns = col_names

        # Clean some columns, Remove Position from Player Name and Remove $ sign from Salary (change to float)
        Table['Player'] =  Table.apply(lambda x: get_name(x['NAME']), axis=1)
        Table['SALARY'] =  (Table['SALARY'].replace( '[\$,)]','', regex=True ).astype(float))
        Table['Contract_Yr'] = season
        Table['Year'] = season - 1   # this field is used for merging to other tables
        
        appended_data.append(Table)
        
    appended_data = pd.concat(appended_data, axis=0)
    return appended_data

##### 用上面函式將 espn.com 中2013~2019年球員的薪水 爬蟲下來 存入回傳並合併dataframe

In [67]:
# List of Tuples - (Year, number of pages)
year_pages = [(2019,11), (2018, 12), (2017, 12), (2016, 11),(2015,11),(2014,10),(2013,14)]

# Append all contract data together
contract_data = pd.concat([scrape_espn_contract(year_page[0], year_page[1])\
                           for year_page in year_pages], ignore_index=True)

In [68]:
contract_data.Contract_Yr.unique()

array([2019, 2018, 2017, 2016, 2015, 2014, 2013])

In [69]:
contract_data.head()

,NAME,TEAM,SALARY,Player,Contract_Yr,Year
0,"Stephen Curry, PG",Golden State Warriors,37457154.0,Stephen Curry,2019,2018
1,"Russell Westbrook, PG",Oklahoma City Thunder,35654150.0,Russell Westbrook,2019,2018
2,"Chris Paul, PG",Houston Rockets,35654150.0,Chris Paul,2019,2018
3,"Blake Griffin, PF",Detroit Pistons,32088932.0,Blake Griffin,2019,2018
4,"Gordon Hayward, SF",Boston Celtics,31214295.0,Gordon Hayward,2019,2018


In [70]:
contract_data.Player.nunique()

1046

In [71]:
contract_data.shape

(3140, 6)

In [72]:
contract_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3140 entries, 0 to 3139
Data columns (total 6 columns):
NAME           3140 non-null object
TEAM           3140 non-null object
SALARY         3140 non-null float64
Player         3140 non-null object
Contract_Yr    3140 non-null int64
Year           3140 non-null int64
dtypes: float64(1), int64(2), object(3)
memory usage: 147.3+ KB


##### 將兩個dataframe存檔並寫入成csv檔 以方便之後分析時直接讀入檔案

In [73]:
import os
os.getcwd()

'/Users/terrylu/Desktop/資料科學程式設計/期末專案/2'

In [74]:
contract_data.to_csv('contract_data')

In [75]:
df_all.to_csv("player_performance_data")

In [76]:
contract=contract_data.drop(['NAME',"Year"],axis = 1 )

In [77]:
contract =contract.rename(columns={'Contract_Yr': 'Year'})

In [78]:
contract.head()

,TEAM,SALARY,Player,Year
0,Golden State Warriors,37457154.0,Stephen Curry,2019
1,Oklahoma City Thunder,35654150.0,Russell Westbrook,2019
2,Houston Rockets,35654150.0,Chris Paul,2019
3,Detroit Pistons,32088932.0,Blake Griffin,2019
4,Boston Celtics,31214295.0,Gordon Hayward,2019


##### 將兩個dataframe合併成新的一個dataframe 存檔並寫入成csv檔 以方便之後分析時直接讀入檔案

In [79]:
df_merge1=pd.merge(df_all,contract)

In [80]:
df_merge1

,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,...,OWS,DWS,WS,WS/48,OBPM,DBPM,BPM,VORP,TEAM,SALARY
0,Quincy Acy,PF,22.0,TOR,29.0,0.0,342.0,4.4,7.9,0.560,...,0.7,0.4,1.1,0.157,-0.6,1.3,0.7,0.2,Toronto Raptors,665000.0
1,Jeff Adrien,PF,26.0,CHA,52.0,5.0,713.0,3.6,8.5,0.429,...,0.5,0.4,1.0,0.064,-3.0,-0.2,-3.3,-0.2,Charlotte Bobcats,916099.0
2,Arron Afflalo,SF,27.0,ORL,64.0,64.0,2307.0,6.2,14.1,0.439,...,1.5,0.5,2.0,0.042,-0.3,-2.0,-2.3,-0.2,Orlando Magic,7750000.0
3,Josh Akognon,PG,26.0,DAL,3.0,0.0,9.0,8.0,16.0,0.500,...,0.0,0.0,0.0,0.196,4.7,-4.9,-0.2,0.0,Dallas Mavericks,473604.0
4,Cole Aldrich,C,24.0,TOT,45.0,0.0,388.0,4.1,7.4,0.550,...,0.1,0.4,0.6,0.070,-4.8,0.7,-4.0,-0.2,Sacramento Kings,2445480.0
5,LaMarcus Aldridge,PF,27.0,POR,74.0,74.0,2790.0,8.2,17.0,0.484,...,4.8,2.3,7.2,0.124,0.3,0.5,0.8,2.0,Portland Trail Blazers,13500000.0
6,Lavoy Allen,C,23.0,PHI,79.0,37.0,1669.0,4.4,9.8,0.454,...,0.9,1.6,2.5,0.072,-3.2,0.6,-2.6,-0.3,Philadelphia 76ers,3000000.0
7,Tony Allen,SG,31.0,MEM,79.0,79.0,2109.0,4.8,10.9,0.445,...,1.0,4.1,5.1,0.115,-1.6,2.9,1.3,1.8,Memphis Grizzlies,3300000.0
8,Al-Farouq Aminu,SF,22.0,NOH,76.0,71.0,2066.0,3.9,8.3,0.475,...,0.7,2.4,3.1,0.073,-2.0,2.4,0.4,1.3,New Orleans Hornets,2947800.0
9,Lou Amundson,PF,30.0,TOT,39.0,0.0,372.0,3.1,7.8,0.395,...,-0.3,0.4,0.1,0.019,-4.4,1.0,-3.3,-0.1,Minnesota Timberwolves,185955.0


In [81]:
#在合併時 有些資料不見了
df_merge1.shape

(2656, 51)

In [82]:
df_merge1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2656 entries, 0 to 2655
Data columns (total 51 columns):
Player    2656 non-null object
Pos       2656 non-null object
Age       2656 non-null float64
Tm        2656 non-null object
G         2656 non-null float64
GS        2656 non-null float64
MP        2656 non-null float64
FG        2656 non-null float64
FGA       2656 non-null float64
FG%       2656 non-null float64
3P        2656 non-null float64
3PA       2656 non-null float64
3P%       2656 non-null float64
2P        2656 non-null float64
2PA       2656 non-null float64
2P%       2656 non-null float64
FT        2656 non-null float64
FTA       2656 non-null float64
FT%       2656 non-null float64
ORB       2656 non-null float64
DRB       2656 non-null float64
TRB       2656 non-null float64
AST       2656 non-null float64
STL       2656 non-null float64
BLK       2656 non-null float64
TOV       2656 non-null float64
PF        2656 non-null float64
PTS       2656 non-null float64


In [84]:
df_merge1.to_csv('df_merge')